In [3]:
import numpy as np
import pandas as pd
import io

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [14]:
data = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [15]:
def preprocess_data(df):
    
    # Feature selection:-
    columns_to_drop =  ['RowNumber','CustomerId','Surname','NumOfProducts']
    df = df.drop(columns=columns_to_drop)
    
    # One-hot encoding categorial columns:-
    geography = pd.get_dummies(df['Geography'],drop_first=True)
    gender = pd.get_dummies(df['Gender'],drop_first=True)
    temp_df = df.drop(['Geography','Gender'],axis=1)
    data = pd.concat([temp_df,geography,gender],axis=1)
    
    return data

data = preprocess_data(data)

In [16]:
data.head()

,CreditScore,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain,Male
0,619,42,2,0.00,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,79084.10,0,0,1,0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [18]:
# Splitting and Feature scaling:

X = data.drop(columns=['Exited'])
y = data['Exited']

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
# Neural Network:-
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical

In [20]:
# Scaling:-
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert X_test to categorical:-
nn_y_test = to_categorical(y_test)
rounded_labels=np.argmax(nn_y_test, axis=1)

# Model

In [21]:
model = Sequential()
model.add(Dense(units=8,kernel_initializer='he_uniform',activation='relu',input_dim=10))
model.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train,y_train,epochs=150,batch_size=64)

Epoch 1/150
125/125 [==============================] - 1s 1ms/step - loss: 0.7325 - accuracy: 0.5232
Epoch 2/150
125/125 [==============================] - 0s 1ms/step - loss: 0.5425 - accuracy: 0.7896
Epoch 3/150
125/125 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7964
Epoch 4/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4895 - accuracy: 0.7923
Epoch 5/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4714 - accuracy: 0.7921
Epoch 6/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4656 - accuracy: 0.7918
Epoch 7/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.7998
Epoch 8/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4426 - accuracy: 0.8052
Epoch 9/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4331 - accuracy: 0.8178
Epoch 10/150
125/125 [==============================] - 0s 1ms/step - loss: 0.4324 - accura

In [23]:
# Saving the Neural Network:
model.save('ANN.h5')

In [24]:
from keras.models import load_model

# Neural Network:-
nn_model = load_model('ANN.h5')
nn_pred = (nn_model.predict(X_test)>0.5).astype(int)

In [25]:
print('The accuracy score for the neural network is: ','\n\n',classification_report(rounded_labels,nn_pred))

The accuracy score for the neural network is:  

               precision    recall  f1-score   support

           0       0.86      0.96      0.91      1607
           1       0.68      0.34      0.45       393

    accuracy                           0.84      2000
   macro avg       0.77      0.65      0.68      2000
weighted avg       0.82      0.84      0.82      2000

